### Data
`df`: premier-league-matches.csv from Kaggle (https://www.kaggle.com/datasets/evangower/premier-league-matches-19922022)

### Goal
Predict full-time results

In [1]:
# load key packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# read in downloaded data

df = pd.read_csv('premier-league-matches.csv')
df

,Season_End_Year,Wk,Date,Home,HomeGoals,AwayGoals,Away,FTR
0,1993,1,1992-08-15,Coventry City,2,1,Middlesbrough,H
1,1993,1,1992-08-15,Leeds United,2,1,Wimbledon,H
2,1993,1,1992-08-15,Sheffield Utd,2,1,Manchester Utd,H
3,1993,1,1992-08-15,Crystal Palace,3,3,Blackburn,D
4,1993,1,1992-08-15,Arsenal,2,4,Norwich City,A
...,...,...,...,...,...,...,...,...
12021,2023,38,2023-05-28,Everton,1,0,Bournemouth,H
12022,2023,38,2023-05-28,Leicester City,2,1,West Ham,H
12023,2023,38,2023-05-28,Aston Villa,2,1,Brighton,H
12024,2023,38,2023-05-28,Leeds United,1,4,Tottenham,A


In [3]:
# check for missings

df.isna().sum()

Season_End_Year    0
Wk                 0
Date               0
Home               0
HomeGoals          0
AwayGoals          0
Away               0
FTR                0
dtype: int64

In [4]:
df.loc[df['Date']=='1992-08-15']

,Season_End_Year,Wk,Date,Home,HomeGoals,AwayGoals,Away,FTR
0,1993,1,1992-08-15,Coventry City,2,1,Middlesbrough,H
1,1993,1,1992-08-15,Leeds United,2,1,Wimbledon,H
2,1993,1,1992-08-15,Sheffield Utd,2,1,Manchester Utd,H
3,1993,1,1992-08-15,Crystal Palace,3,3,Blackburn,D
4,1993,1,1992-08-15,Arsenal,2,4,Norwich City,A
5,1993,1,1992-08-15,Ipswich Town,1,1,Aston Villa,D
6,1993,1,1992-08-15,Everton,1,1,Sheffield Weds,D
7,1993,1,1992-08-15,Southampton,0,0,Tottenham,D
8,1993,1,1992-08-15,Chelsea,1,1,Oldham Athletic,D


Modify the dataframe to have one row for each team as the main team rather than Home and Away on one line

In [5]:
# create a copy of the dataframe
df_copy = df.copy()

In [6]:
# mod the copy column names
df_copy.rename(columns = {'Away': 'team',
                         'Home': 'opponent',
                         'HomeGoals':'ga',
                         'AwayGoals':'gf',
                         'Season_End_Year':'season',
                         'Wk':'wk',
                         'Date':'date',
                         'FTR':'ftr'},
               inplace = True)
# add home or away column
df_copy['ha'] = 'A'

# modify the ftr column
df_copy['ftr'] = df_copy['ftr'].replace({'H': 'L', 'A': 'W'})

# reorder columns
df_copy = df_copy[['season', 'wk', 'date', 'ha', 'team', 'opponent', 'gf', 'ga', 'ftr']]

df_copy

,season,wk,date,ha,team,opponent,gf,ga,ftr
0,1993,1,1992-08-15,A,Middlesbrough,Coventry City,1,2,L
1,1993,1,1992-08-15,A,Wimbledon,Leeds United,1,2,L
2,1993,1,1992-08-15,A,Manchester Utd,Sheffield Utd,1,2,L
3,1993,1,1992-08-15,A,Blackburn,Crystal Palace,3,3,D
4,1993,1,1992-08-15,A,Norwich City,Arsenal,4,2,W
...,...,...,...,...,...,...,...,...,...
12021,2023,38,2023-05-28,A,Bournemouth,Everton,0,1,L
12022,2023,38,2023-05-28,A,West Ham,Leicester City,1,2,L
12023,2023,38,2023-05-28,A,Brighton,Aston Villa,1,2,L
12024,2023,38,2023-05-28,A,Tottenham,Leeds United,4,1,W


In [7]:
# repeat process with the original dataframe
df.rename(columns = {'Home': 'team',
                         'Away': 'opponent',
                         'AwayGoals':'ga',
                         'HomeGoals':'gf',
                         'Season_End_Year':'season',
                         'Wk':'wk',
                         'Date':'date',
                         'FTR':'ftr'},
               inplace = True)
# add home or away column
df['ha'] = 'H'

# modify the 'ftr' column
df['ftr'] = df['ftr'].replace({'H': 'W', 'A': 'D'})

# reorder columns
df = df[['season', 'wk', 'date', 'ha', 'team', 'opponent', 'gf', 'ga', 'ftr']]

df

,season,wk,date,ha,team,opponent,gf,ga,ftr
0,1993,1,1992-08-15,H,Coventry City,Middlesbrough,2,1,W
1,1993,1,1992-08-15,H,Leeds United,Wimbledon,2,1,W
2,1993,1,1992-08-15,H,Sheffield Utd,Manchester Utd,2,1,W
3,1993,1,1992-08-15,H,Crystal Palace,Blackburn,3,3,D
4,1993,1,1992-08-15,H,Arsenal,Norwich City,2,4,D
...,...,...,...,...,...,...,...,...,...
12021,2023,38,2023-05-28,H,Everton,Bournemouth,1,0,W
12022,2023,38,2023-05-28,H,Leicester City,West Ham,2,1,W
12023,2023,38,2023-05-28,H,Aston Villa,Brighton,2,1,W
12024,2023,38,2023-05-28,H,Leeds United,Tottenham,1,4,D


In [8]:
# concat dataframes
df_a = pd.concat([df, df_copy])

# sort values
df_a = df_a.reset_index()
df_a.sort_values(by='index', inplace=True)

# reset index again
df_a = df_a.reset_index()

# drop old index columns
df_a.drop(columns = ['level_0', 'index'], inplace=True)

df_a

,season,wk,date,ha,team,opponent,gf,ga,ftr
0,1993,1,1992-08-15,H,Coventry City,Middlesbrough,2,1,W
1,1993,1,1992-08-15,A,Middlesbrough,Coventry City,1,2,L
2,1993,1,1992-08-15,A,Wimbledon,Leeds United,1,2,L
3,1993,1,1992-08-15,H,Leeds United,Wimbledon,2,1,W
4,1993,1,1992-08-15,H,Sheffield Utd,Manchester Utd,2,1,W
...,...,...,...,...,...,...,...,...,...
24047,2023,38,2023-05-28,H,Aston Villa,Brighton,2,1,W
24048,2023,38,2023-05-28,A,Tottenham,Leeds United,4,1,W
24049,2023,38,2023-05-28,H,Leeds United,Tottenham,1,4,D
24050,2023,38,2023-05-28,H,Brentford,Manchester City,1,0,W


In [10]:
# make sure teams are named uniformly
df_a.team.value_counts()

team
Everton            1190
Liverpool          1190
Chelsea            1190
Manchester Utd     1190
Arsenal            1190
Tottenham          1190
Aston Villa        1076
Newcastle Utd      1072
West Ham           1034
Manchester City    1000
Southampton         924
Blackburn           696
Leicester City      650
Sunderland          608
Fulham              608
Leeds United        582
Middlesbrough       574
Crystal Palace      540
Bolton              494
West Brom           494
Norwich City        392
Stoke City          380
Coventry City       354
Wolves              342
Wimbledon           316
Sheffield Weds      316
Burnley             304
Wigan Athletic      304
Charlton Ath        304
Watford             304
QPR                 278
Birmingham City     266
Portsmouth          266
Swansea City        266
Derby County        266
Nott'ham Forest     236
Brighton            228
Bournemouth         228
Ipswich Town        202
Sheffield Utd       198
Hull City           190
Reading    

In [12]:
# add day of the week column
# convert 'date' column type
df_a['date'] = pd.to_datetime(df_a['date'])

# add abbreviated day of week column
df_a['day'] = df_a['date'].dt.strftime('%a')

# reorder columns
df_a = df_a[['season', 'wk', 'date', 'day', 'ha', 'team', 'opponent', 'gf', 'ga', 'ftr']]

df_a

,season,wk,date,day,ha,team,opponent,gf,ga,ftr
0,1993,1,1992-08-15,Sat,H,Coventry City,Middlesbrough,2,1,W
1,1993,1,1992-08-15,Sat,A,Middlesbrough,Coventry City,1,2,L
2,1993,1,1992-08-15,Sat,A,Wimbledon,Leeds United,1,2,L
3,1993,1,1992-08-15,Sat,H,Leeds United,Wimbledon,2,1,W
4,1993,1,1992-08-15,Sat,H,Sheffield Utd,Manchester Utd,2,1,W
...,...,...,...,...,...,...,...,...,...,...
24047,2023,38,2023-05-28,Sun,H,Aston Villa,Brighton,2,1,W
24048,2023,38,2023-05-28,Sun,A,Tottenham,Leeds United,4,1,W
24049,2023,38,2023-05-28,Sun,H,Leeds United,Tottenham,1,4,D
24050,2023,38,2023-05-28,Sun,H,Brentford,Manchester City,1,0,W


In [14]:
# save data to csv file
df_a.to_csv('prem_league_data_processed.csv')